# Template Attack: Introduction, Hamming Weight Assumption

*Template attacks* are a powerful type of side-channel attack. These attacks are a subset of *profiling attacks*, where an attacker creates a "profile" of a sensitive device and applies this profile to quickly find a victim's secret key. 

Template attacks require more setup than CPA attacks. To perform a template attack, the attacker must have access to another copy of the protected device that they can fully control. Then, they must perform a great deal of pre-processing to create the template - in practice, this may take dozens of thousands of power traces. However, the advantages are that template attacks require a very small number of traces from the victim to complete the attack. With enough pre-processing, the key may be able to be recovered from just a single trace. 

There are four steps to a template attack:
1. Using a copy of the protected device, record a large number of power traces using many different inputs (plaintexts and keys). Ensure that enough traces are recorded to give us information about each subkey value.
2. Create a template of the device's operation. This template notes a few "points of interest" in the power traces and a multivariate distribution of the power traces at each point. 
3. On the victim device, record a small number of power traces. Use multiple plaintexts. (We have no control over the secret key, which is fixed.)
4. Apply the template to the attack traces. For each subkey, track which value is most likely to be the correct subkey. Continue until the key has been recovered.

## Signals, Noise, and Statistics
Before looking at the details of the template attack, it is important to understand the statistics concepts that are involved. A template is effectively a multivariate distribution that describes several key samples in the power traces. This section will describe what a multivariate distribution is and how it can be used in this context.

### Noise Distributions
Electrical signals are inherently noisy. Any time we take a voltage measurement, we don't expect to see a perfect, constant level. For example, if we attached a multimeter to a 5 V source and took 4 measurements, we might expect to see a data set like (4.95, 5.01, 5.06, 4.98). One way of modelling this voltage source is:

$$\mathbf{X} = X_{actual} + \mathbf{N}$$

A simple model for these random variables uses a Gaussian distribution (read: a bell curve). The probability density function (PDF) of a Gaussian distribution is

$$f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-(x - \mu)^2 / 2\sigma^2}$$

where $\mu$ is the mean and $\sigma$ is the standard deviation. For instance, our voltage source might have a mean of 5 and a standard deviation of 0.5. We can plot this curve:

In [ ]:
from scipy import stats
from matplotlib import pylab as plt
import numpy as np
#help(stats)

mu = 5
sigma = 0.5

x = np.linspace(3, 7, 5000)
y_pdf = stats.norm.pdf(x, mu, sigma)

plt.plot(x, y_pdf, label='pdf')
plt.title("PDF of $\mu = %f$, $\sigma = %f$"%(mu, sigma))
plt.xlabel("Random Variable Value")
plt.ylabel("Density Value")
plt.show()

What's this graph all about? This graph shows the *probability density* of this random variable. Note it does NOT give you an actual probability, instead you need to integrate to find the probability the variable is lieing in some range. This is important because the density function won't always return just < 1, and with multivariate (to be discussed) the output is frequently larger than one. You can find probabilities using the CDF, which integrates from negative infinity to the given random variable value.

For example, what is the chance that we will observe a value in the range 6.0?

In [ ]:
pval = stats.norm.cdf(6.0, mu, sigma)
print("%.2f %%"%(pval*100))

PDFs help us answer a simple question (that will be relevant to us shortly). Let's say we have the following experiment setup:


We have two batteries and are trying to figure out which one is connected behind the curtain. There is a *lot* of noise in our measurement setup and we don't have much time to waste, so can only measure a few points. The following will simulate a number of measurements from one or the other battery, but not tell us which one is which.

In [ ]:
from random import randint
import numpy as np

noise = 2.0

def do_measure():
    if randint(0,1):
        return (1.5, np.random.normal(1.5, noise))
    else:
        return (3.0, np.random.normal(3.0, noise))
 
N = 10

measure_array = [0]*N
answer = [0]*N

for n in range(0, N):
    answer[n], measure_array[n] = do_measure()
    print("%.02f"%measure_array[n])
    

Let's see how you stack up to a computer! Try entering your guess of the measurements here by editing the values of the array to reflect the printed measurements. Select if you think the source was 1.5V or 3.0V.

In [ ]:
guess = [1.5, 1.5, 3.0, 3.0, 1.5, 1.5, 3.0, 1.5, 3.0, 3.0]

How does the PDF and CDF information help us? We could use the PDF to provide some information on the liklihood of a specific input. Note this should be done with another function (remember the PDF is a density function, not a probability function), but we will use PDF for now as the template attack to be discussed will continue this method.

In [ ]:
import pandas as pd

results = []*N

for n in range(0, N):
    p15 = stats.norm.pdf(measure_array[n], 1.5, noise)
    p30 = stats.norm.pdf(measure_array[n], 3.0, noise)

    if p15 > p30:
        bguess = 1.5
    else:
        bguess = 3.0
        
    results.append([p15, p30, bguess, guess[n], answer[n]])
    
pd.DataFrame(results, columns=["P|m=1.5", "P|m=3.0", "Best Guess", "Your Guess", "Answer"])

### Multivariate Statistics

The 1-variable Gaussian distribution works well for one measurement. What if we're working with more than one random variable?

Suppose we're measuring two voltages that have some amount of noise on them. We'll call them $\mathbf{X}$ and $\mathbf{Y}$. As a first attempt, we could write down a model for $\mathbf{X}$ using a normal distribution and a separate model for $\mathbf{Y}$ using a different distribution. However, this might not always make sense. If we write two separate distributions, what we're saying is that the two variables are independent: when $\mathbf{X}$ goes up, there's no guarantee that $\mathbf{Y}$ will follow it.

Multivariate distributions let us model multiple random variables that may or may not be correlated. In a multivariate distribution, instead of writing down a single variance $\sigma$, we keep track of a whole matrix of covariances. For example, to model three random variables ($\mathbf{X}, \mathbf{Y}, \mathbf{Z}$), this matrix would be

$$
\mathbf{\Sigma} = 
\begin{bmatrix}
Var(\mathbf{X})             & Cov(\mathbf{X}, \mathbf{Y}) & Cov(\mathbf{X}, \mathbf{Z}) \\
Cov(\mathbf{Y}, \mathbf{X}) & Var(\mathbf{Y})             & Cov(\mathbf{Y}, \mathbf{Z}) \\
Cov(\mathbf{Z}, \mathbf{X}) & Cov(\mathbf{Z}, \mathbf{Y}) & Var(\mathbf{Z}) 
\end{bmatrix}
$$

Also, note that this distribution needs to have a mean for each random variable:

$$
\mathbf{\mu} = 
\begin{bmatrix}
\mu_X \\
\mu_Y \\
\mu_Z
\end{bmatrix}
$$

The PDF of this distribution is more complicated: instead of using a single number as an argument, it uses a vector with all of the variables in it ($\mathbf{x} = [x, y, z, \dots]^T$). The equation for $k$ random variables is

$$
f(\mathbf{x})
= \frac{1}{\sqrt{(2\pi)^k |\mathbf{\Sigma}|}} 
  e^{-(\mathbf{(x - \mu)}^T \mathbf{\Sigma}^{-1} \mathbf{(x - \mu)} / 2}
$$

Don't worry if this looks crazy - the SciPy package in Python will do all the heavy lifting for us. As with the single-variable distributions, we're going to use this to find how likely a certain observation is. In other words, if we put $k$ points of our power trace into $\mathbf{x}$ and we find that $f(\mathbf{x})$ is very high, then we've probably found a good guess.


## Creating the Template

A template is a set of probability distributions that describe what the
power traces look like for many different keys. Effectively, a template
says: \"If you\'re going to use key $k$, your power trace will look like
the distribution $f_k(\mathbf{x})$\". We can use this information to
find subtle differences between power traces and to make very good key
guesses for a single power trace.

### Number of Traces

One of the downsides of template attacks is that they require a great
number of traces to be preprocessed before the attack can begin. This is
mainly for statistical reasons. In order to come up with a good
distribution to model the power traces for *every key*, we need a large
number of traces for *every key*. For example, if we\'re going to attack
a single subkey of AES-128, then we need to create 256 power consumption
models (one for every number from 0 to 255). In order to get enough data
to make good models, we need tens of thousands of traces.

Note that we don\'t have to model every single key. One good alternative
is to model a sensitive part of the algorithm, like the substitution box
in AES. We can get away with a much smaller number of traces here; if we
make a model for every possible Hamming weight, then we would end up
with 9 models, which is an order of magnitude smaller. However, then we
can\'t recover the key from a single attack trace - we need more
information to recover the secret key.

### Points of Interest

Our goal is to create a multivariate probability describing the power
traces for every possible key. If we modeled the entire power trace this
way (with, say, 3000 samples), then we would need a 3000-dimension
distribution. This is insane, so we\'ll find an alternative.

Thankfully, not every point on the power trace is important to us. There
are two main reasons for this:

-   We might be taking more than one sample per clock cycle. (Through
    most of the ChipWhisperer tutorials, our ADC runs four times faster
    than the target device.) There\'s no real reason to use all of these
    samples - we can get just as much information from a single sample
    at the right time.
-   Our choice of key doesn\'t affect the entire power trace. It\'s
    likely that the subkeys only influence the power consumption at a
    few critical times. If we can pick these important times, then we
    can ignore most of the samples.

These two points mean that we can usually live with a handful (3-5) of
*points of interest*. If we can pick out good points and write down a
model using these samples, then we can use a 3D or 5D distribution - a
great improvement over the original 3000D model.

#### Picking POIs

There are several ways to pick the most important points in each of the
traces. Generally, the aim is to find points that vary strongly between
different operations (subkeys or Hamming weights). The simplest method
\-- the one that we\'ll use here \-- is the *sum of differences* method.

The algorithm for the sum of difference method is:

-   For every operation $k$ and every sample $i$, find the average power
    $M_{k, i}$. For instance, if there are $T_k$ traces where we
    performed operation $k$, then this average is

$M_{k, i} = \frac{1}{T_k} \sum_{j=1}^{T_k} t_{j, i}$

-   After finding all of the means, calculate all of their absolute
    pairwise differences. Add these up. This will give one \"trace\"
    which has peaks where the samples are usually different. The
    calculation looks like

$D_{i} = \sum_{k_1, k_2} |M_{k_1, i} - M_{k_2, i}|$

An example of this sum of differences is:

![][1]

-   The peaks of $D_i$ show the most important points, but we need to
    satisfy point 1 from above - we need to pick some peaks that aren\'t
    too close. One algorithm to do this is:

1.  Pick the highest point in $D_i$ and save this value of $i$ as a
    point of interest. (ie: $i = argmax(D_i)$)
2.  Throw out the nearest $N$ points (where $N$ is the minimum spacing
    between POIs).
3.  Repeat until enough POIs have been selected.

### Analyzing the Data

Suppose that we\'ve picked $I$ points of interest, which are at samples
$s_i$ ($0 \le i < I$). Then, our goal is to find a mean and covariance
matrix for every operation (every choice of subkey or intermediate
Hamming weight). Let\'s say that there are $K$ of these operations
(maybe 256 subkeys or 9 possible Hamming weights).

For now, we\'ll look at a single operation $k$ ($0 \le k < K$). The
steps are:

-   Find every power trace $t$ that falls under the category of
    \"operation $k$\". (ex: find every power trace where we used a
    subkey of 0x01.) We\'ll say that there are $T_k$ of these, so
    $t_{j, s_i}$ means the value at trace $j$ and POI $i$.
-   Find the average power $\mu_i$ at every point of interest. This
    calculation will look like:

$\mu_i = \frac{1}{T_k} \sum_{j=1}^{T_k} t_{j, s_i}$

-   Find the variance $v_i$ of the power at each point of interest. One
    way of calculating this is:

$v_i = \frac{1}{T_k} \sum_{j=1}^{T_k} (t_{j, s_i} - \mu_i)^2$

-   Find the covariance $c_{i, i^*}$ between the power at every pair of
    POIs ($i$ and $i^*$). One way of calculating this is:

$c_{i, i^*} = \frac{1}{T_k}  \sum_{j=1}^{T_k} (t_{j, s_i} - \mu_i) (t_{j, s_{i^*}} - \mu_{i^*})$

-   Put together the mean and covariance matrices as:

$\mathbf{\mu} = 
\begin{bmatrix}
\mu_1 \\
\mu_2 \\
\mu_3 \\
\vdots
\end{bmatrix}$

$\mathbf{\Sigma} = 
\begin{bmatrix}
v_1     & c_{1,2} & c_{1,3} & \dots \\
c_{2,1} & v_2     & c_{2,3} & \dots \\
c_{3,1} & c_{3,2} & v_3     & \dots \\
\vdots  & \vdots  & \vdots  & \ddots 
\end{bmatrix}$

These steps must be done for every operation $k$. At the end of this
preprocessing, we\'ll have $K$ mean and covariance matrices, modelling
each of the $K$ different operations that the target can do.

Using the Template
------------------

With a template in hand, we can finish our attack. For the attack, we
need a smaller number of traces - we\'ll say that we have $A$ traces.
The sample values will be labeled $a_{j, s_i}$ ($1 \le j \le A$).

### Applying the Template

First, let\'s apply the template to a single trace. Our job is to decide
how likely all of our key guesses are. We need to do the following:

-   Put our trace values at the POIs into a vector. This vector will be

$\mathbf{a_j} = 
\begin{bmatrix}
a_{j,1} \\
a_{j,2} \\
a_{j,3} \\
\vdots
\end{bmatrix}$

-   Calculate the PDF for every key guess and save these for later. This
    might look like:

$p_{k, j} = f_k(\mathbf{a_j})$

-   Repeat these two steps for all of the attack traces.

This process gives us an array of $p_{k, j}$, which says: \"Looking at
trace $j$, how likely is it that key $k$ is the correct one?\"

### Combining the Results

The very last step is to combine our $p_{k, j}$ values to decide which
key is the best fit. The easiest way to do this is to combine them as

$P_k = \prod_{j=1}^{A} p_{k,j}$

For example, if we guessed that a subkey was equal to 0x00 and our PDF
results in 3 traces were (0.9, 0.8, 0.95), then our overall result would
be 0.684. Having one trace that doesn\'t match the template can cause
this number to drop quickly, helping us eliminate the wrong guesses.
Finally, we can pick the highest value of $P_k$, which tells us which
guess fits the templates the best, and we\'re done!

This method of combining our per-trace results can suffer from precision
issues. After multiplying many large or small numbers together, we could
end up with numbers that are too large or small to fit into a floating
point variable. An easy fix is to work with logarithms. Instead of using
$P_k$ directly, we can calculate

$\log P_k = \sum_{j=1}^A \log p_{k,j}$

Comparing these logarithms will give us the same results without the
precision issues.

  [1]: Template-Sum-Of-Difference.png "Template-Sum-Of-Difference.png"
  {width="600"}

## Capturing Power Traces

In [ ]:
%run "Helper_Scripts/CWLite_Connect.ipynb"

In [ ]:
%run "Helper_Scripts/Setup_Target_Generic.ipynb"

In [ ]:
# uncomment based on your target
#%run "Helper_Scripts/Program_XMEGA.ipynb"
%run "Helper_Scripts/Program_STM.ipynb"
#%run "Helper_Scripts/No_Programmer.ipynb"
fw_path = "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-cwlitearm.hex"
#fw_path = "../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-xmega.hex"

In [ ]:
# program the target
program_target(scope, fw_path)

### Capturing Template (Rand Key, Rand Text)

The first capture will be for the template. This template requires us to have full control of the device, both the key and plaintext. We will be using a **random key**, meaning the key will change on **every** encryption.

Why would we do that? The reason is that if we used a **fixed key** during the profile, there is a strong chance that we will "overfit" the templates. That is the templates will only be valid for the fixed key we used during the profiling phase. Obviously this won't make for a useful attack in practice, since we don't know the actual key in use.

Our "overfit" model would also mean our results appear to work increadily well, but they won't work in a real situation. That is, should you perform profiling and evaluation with the same fixed key, you will get very good results due to the overfit but they are meaningless in practice.

The capture phase is otherwise similar to before. The difference here is that we will have to tell the acquisition model that we want to use a random key. The following simple block will create a new project and save 2000 traces.

Remember that the resulting group of Hamming weights won't be uniformally distributed. This will affect our template results, as for example the group with a Hamming weight of 8 will have only about $1/256 * 2000 = 8$ traces. This is because the Hamming weight of 8 only exists for 1/256 possible values of the S-Box output. Hamming weights of 3,4,5 are much more likely for example and will contain much larger trace groups.

In [ ]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import chipwhisperer as cw
import numpy as np
import time

#Make a project to save template traces
project = cw.createProject("projects/tutorial_template_templatedata.cwp", overwrite = True)

ktp = AcqKeyTextPattern_Basic(target=target)
ktp.setKeyType(False) #RANDOM KEY in addition to RANDOM TEXT

#Setup this segment with default information
tc = project.getNewTraceSegment()

N = 5000  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    textout = target.readOutput()  # Response
    tc.addTrace(scope.getLastTrace(), text, textout, key)
    
#End this segment, and add to project
tc.closeAll()
project.traceManager().appendSegment(tc)

#Save the project
project.save()

While we're here - why not capture the traces for the attack too. These traces will be much shorter, we'll only capture 20 traces and probably not even need all of them.

In [ ]:
#Capture Traces
from tqdm import tqdm
from chipwhisperer.capture.acq_patterns.basic import AcqKeyTextPattern_Basic
import chipwhisperer as cw
import numpy as np
import time

#Make a project to save test traces
project = cw.createProject("projects/tutorial_template_validate.cwp", overwrite = True)

ktp = AcqKeyTextPattern_Basic(target=target)

#Setup this segment with default information
tc = project.getNewTraceSegment()

N = 20  # Number of traces
target.init()
for i in tqdm(range(N), desc='Capturing traces'):
    # run aux stuff that should come before trace here

    key, text = ktp.newPair()  # manual creation of a key, text pair can be substituted here

    #target.reinit()

    target.setModeEncrypt()  # only does something for targets that support it
    target.loadEncryptionKey(key)
    target.loadInput(text)

    # run aux stuff that should run before the scope arms here

    scope.arm()

    # run aux stuff that should run after the scope arms here

    target.go()
    timeout = 50
    # wait for target to finish
    while target.isDone() is False and timeout:
        timeout -= 1
        time.sleep(0.01)

    try:
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')
    except IOError as e:
        print('IOError: %s' % str(e))

    textout = target.readOutput()  # Response
    tc.addTrace(scope.getLastTrace(), text, textout, key)
    
#End this segment, and add to project
tc.closeAll()
project.traceManager().appendSegment(tc)

#Save the project
project.save()

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## Building the Templates

### Opening & Grouping Traces

In [ ]:
import chipwhisperer as cw
project_template = cw.openProject("projects/tutorial_template_templatedata.cwp")

In [ ]:
tm_template = project_template.traceManager()

#Let's confirm that we get random keys
for i in range(0, 4):
    print("%d: "%i + " ".join(["%02x"%tm_template.getKnownKey(i)[j] for j in range(0, 16)]))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(tm_template.getTrace(0))
plt.show()

intermediate = sbox[plaintext ^ key]

In [ ]:
sbox=(
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16) 

hw = [bin(x).count("1") for x in range(256)]

for n in range(0, 10):
    tin = tm_template.getTextin(n)[0]
    key = tm_template.getKnownKey(n)[0]
    
    intermediate = sbox[tin ^ key]
    
    print("Trace %d: Textin[0] = %02x  Key[0] = %02x --> SBox Output = %02x --> HW = %d"%(
        n, tin, key, intermediate, hw[intermediate]))
    

In [ ]:
def cov(x, y):
    # Find the covariance between two 1D lists (x and y).
    # Note that var(x) = cov(x, x)
    return np.cov(x, y)[0][1]


Next, we apply that across all the traces in the profiling run. This will let us "split" each trace into a different group according to the resulting Hamming weights:

In [ ]:
# 2: Find HW(sbox) to go with each input
# Note - we're only working with ONE byte here
target_byte = 0
tempSbox = [sbox[tm_template.getTextin(n)[target_byte] ^ tm_template.getKnownKey(n)[target_byte]] for n in range(tm_template.numTraces())] 
tempHW   = [hw[s] for s in tempSbox]

# 2.5: Sort traces by HW
# Make 9 blank lists - one for each Hamming weight
tempTracesHW = [[] for _ in range(9)]

# Fill them up
for i in range(tm_template.numTraces()):
    HW = tempHW[i]
    tempTracesHW[HW].append(tm_template.getTrace(i))

# Switch to numpy arrays
tempTracesHW = [np.array(tempTracesHW[HW]) for HW in range(9)]

#print len(tempTracesHW[8])

### Points of Interest

After sorting the traces by their Hamming weights, we need to find an "average trace" for each weight. We can make an array to hold 9 of these averages:

```
tempMeans = np.zeros((9, len(tempTraces[0])))
```

Then, we can fill up each of these traces one-by-one. NumPy's <code>average()</code> function makes this easy by including an "axis" input. We can tell NumPy to find the average at each point in time and repeat this for all 9 weights:

```
for i in range(9):
    tempMeans[i] = np.average(tempTracesHW[i], 0)
```

Once again, it's a good idea to plot one of these averages to make sure that the average traces look okay:

In [ ]:
# 3: Find averages
tempMeans = np.zeros((9, tm_template.numPoints()))
for i in range(9):
    tempMeans[i] = np.average(tempTracesHW[i], 0)
    
plt.plot(tempMeans[2])
plt.grid()
plt.show()

We can use these average traces to find points of interest using the ''sum of differences'' method. The first step is to create a "trace" that stores these differences:

```
tempSumDiff = np.zeros(len(tempTraces[0]))
```

Then, we want to look at all of the pairs of traces, subtract them, and add them to the sum of differences. This is simple to do with 2 loops:

```
for i in range(9):
    for j in range(i):
        tempSumDiff += np.abs(tempMeans[i] - tempMeans[j])
```

This sum of differences will have peaks where the average traces showed a lot of variance. Make sure that there are a handful of high peaks that we can use as points of interest:

In [ ]:
# 4: Find sum of differences
tempSumDiff = np.zeros(tm_template.numPoints())
for i in range(9):
    for j in range(i):
        tempSumDiff += np.abs(tempMeans[i] - tempMeans[j])

plt.plot(tempSumDiff)
plt.grid()
plt.show()

Now, the most interesting points of interest are the highest peaks in this sum of differences plot. However, we can't just sort the array and pick the top points. Remember (because you read the theory page, right? <sup>right?</sup>) that we need to make sure our points have some space between them. For instance, it would be a bad idea to pick both 1950 and 1951 as points of interest.

We can use the algorithm from the theory page to pick out some POIs:

* Make an empty list of POIs
* Find the biggest peak in the sum of differences trace and add it to the list of POIs
* Zero out some of the surrounding points
* Repeat until we have enough POIs

Try to code this on your own - it's a good exercise. If you get stuck, here's our implementation:

In [ ]:
# 5: Find POIs
POIs = []
numPOIs = 5
POIspacing = 5
for i in range(numPOIs):
    # Find the max
    nextPOI = tempSumDiff.argmax()
    POIs.append(nextPOI)
    
    # Make sure we don't pick a nearby value
    poiMin = max(0, nextPOI - POIspacing)
    poiMax = min(nextPOI + POIspacing, len(tempSumDiff))
    for j in range(poiMin, poiMax):
        tempSumDiff[j] = 0
    
#print POIs

In [ ]:
plt.plot(tempSumDiff)

In [ ]:
print(POIs)

### Generating a Template

With 5 (or <code>numPOIs</code>) POIs picked out, we can build our multivariate distributions at each point for each Hamming weight. We need to write down two matrices for each weight:

* A mean matrix (<code>1 x numPOIs</code>) which stores the mean at each POI
* A covariance matrix (<code>numPOIs x numPOIs</code>) which stores the variances and covariances between each of the POIs

The mean matrix is easy to set up because we've already found the mean at every point. All we need to do is grab the right points:

```
meanMatrix = np.zeros((9, numPOIs))
for HW in range(9):
    for i in range(numPOIs):
        meanMatrix[HW][i] = tempMeans[HW][POIs[i]]
```

The covariance matrix is a bit more complex. We need a way to find the covariance between two 1D arrays. Helpfully, NumPy has the <code>cov(a, b)</code> function, which returns the matrix
```
np.cov(a, b) = [[cov(a, a), cov(a, b)],
                [cov(b, a), cov(b, b)]]
```

We can use this to define our own covariance function:
```
def cov(x, y):
    # Find the covariance between two 1D lists (x and y).
    # Note that var(x) = cov(x, x)
    return np.cov(x, y)[0][1]
```

As mentioned in the comments, this function can also calculate the variance of an array by passing the same array in for both <code>x</code> and <code>y</code>.

We'll use our function to build the covariance matrices, as below:

In [ ]:
# 6: Fill up mean and covariance matrix for each HW
meanMatrix = np.zeros((9, numPOIs))
covMatrix  = np.zeros((9, numPOIs, numPOIs))
for HW in range(9):
    for i in range(numPOIs):
        # Fill in mean
        meanMatrix[HW][i] = tempMeans[HW][POIs[i]]
        for j in range(numPOIs):
            x = tempTracesHW[HW][:,POIs[i]]
            y = tempTracesHW[HW][:,POIs[j]]
            covMatrix[HW,i,j] = cov(x, y)

Finally - let's confirm these matricies look A-OK. Basically you need to ensure there isn't zeros in them, which normally indiciates you didn't have enough data.

In [ ]:
print(meanMatrix)
print(covMatrix[0])

### Applying the Template

The very last step is to apply our template to these traces. We want to keep a running total of $\log P_k = \sum_j \log p_{k,j}$, so we'll make space for our 256 guesses:
```
P_k = np.zeros(256)
```

Then, we want to do the following for every attack trace:

* Grab the samples from the points of interest and store them in a list $\mathbf{a}$
* For all 256 of the subkey guesses...
 * Figure out which Hamming weight we need, according to our known plaintext and guessed subkey
 * Build a <code>multivariate_normal</code> object using the relevant mean and covariance matrices
 * Calculate the log of the PDF ($\log f(\mathbf{a})$) and add it to the running total
* List the best guesses we've seen so far

Make sure you've included the multivariate stats from SciPy too - it's normally an extra import.

In [ ]:
import chipwhisperer as cw
project_validate = cw.openProject("projects/tutorial_template_validate.cwp")
tm_validate = project_validate.traceManager()

In [ ]:
from scipy.stats import multivariate_normal
# 2: Attack
# Running total of log P_k
P_k = np.zeros(256)
for j in range(tm_validate.numTraces()):
    # Grab key points and put them in a small matrix
    a = [tm_validate.getTrace(j)[POIs[i]] for i in range(len(POIs))]
    
    # Test each key
    for k in range(256):
        # Find HW coming out of sbox
        HW = hw[sbox[tm_validate.getTextin(j)[target_byte] ^ k]]
    
        # Find p_{k,j}
        rv = multivariate_normal(meanMatrix[HW], covMatrix[HW])
        p_kj = rv.logpdf(a)
   
        # Add it to running total
        P_k[k] += p_kj

    # Print our top 5 results so far
    # Best match on the right
    print(" ".join(["%02x"%j for j in P_k.argsort()[-5:]]))

With any luck you will have found the correct key byte! You can try extending this to attack multiple bytes instead. The traces you already recorded will work just great, you'll need to make the following changes:

* Build a new POI for each byte number
* Build new matricies for each byte number
* Apply those matricies for each byte

## Conclusions

That's it! This template attack must have raised some new questions for you. A few things to try beyond the above:

* Get rid of the Hamming weight assumption. You can build templates based on each key byte value itself directly, which means a SINGLE power trace would result in the secret key(!).
 * Doing this requires a *lot* more data, since you need to build information on each key byte.
 * Make your life easier by using a FIXED plaintext. The same plaintext is used during building the template as when you apply the attack.